# Neural net speech recognition and summarization for Videos
Loading libraries and global variables

In [1]:
import subprocess
import moviepy.editor as mp
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import whisper
# https://openai.com/blog/whisper/
# pip install git+https://github.com/openai/whisper.git, not whisper.py unrelated package
from pathlib import Path
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch
import os

model_name = 'Falconsai/text_summarization'#'google/pegasus-xsum' #'facebook/bart-large-cnn'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)
# https://arxiv.org/pdf/1912.08777.pdf
# https://medium.com/codex/build-an-automatic-abstractive-text-summarizer-in-ten-minutes-f15f07e54bae

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'PegasusTokenizer'.
You are using a model of type t5 to instantiate a model of type pegasus. This is not supported for all configurations of models and can yield errors.
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at Falconsai/text_summarization and are newly initialized: ['decoder.embed_positions.weight', 'decoder.layer_norm.bias', 'decoder.layer_norm.weight', 'decoder.layers.0.encoder_attn.k_proj.bias', 'decoder.layers.0.encoder_attn.k_proj.weight', 'decoder.layers.0.encoder_attn.out_proj.bias', 'decoder.layers.0.encoder_attn.out_proj.weight', 'decoder.layers.0.encoder_attn.q_proj.bias', 'decoder.layers.0.encoder_attn.q_proj.weight', 'decoder.layers.0.encoder_attn.v_proj

Función para determinar la duración total del video

In [2]:
def get_length(filename):
    result = subprocess.run(["ffprobe", "-v", "error", "-show_entries",
                             "format=duration", "-of",
                             "default=noprint_wrappers=1:nokey=1", filename],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT)
    return float(result.stdout)

Function to generate the video subtitles, separating them into blocks of chunk_size seconds. The function can be limited so that it only generates the subtitles between the start and stop seconds, and can be used as:

-VideoToSRT("video.mp4")

-VideoToSRT("video.mp4", 10)

-VideoToSRT("video.mp4", 10, 52)

-VideoToSRT("video.mp4", 5, 34, 212)

In [3]:
def VideoToSRT(filename, chunk_size=5, start=0, stop=-1):
    num_seconds_video= int(get_length(filename))+1
    print("The video is {} seconds".format(num_seconds_video))
    l=list(range(0,num_seconds_video+1,chunk_size))
    rank=range(len(l)-1)
    if stop>num_seconds_video:
        print("Upper time limit exceeds the duration of the video, it will be considered the end of the video")
    if start>0:
        emp=int(start/chunk_size)
    else:
        emp=0
    if stop>-1:
        term=int(stop/chunk_size)+1
    else:
        term=len(l)-1
    rank=range(emp,term)
    diz={}

    for i in rank:
        ffmpeg_extract_subclip(filename, l[i]-2*(l[i]!=0), l[i+1], targetname="chunks/cut{}.mp4".format(i+1))
        clip = mp.VideoFileClip(r"chunks/cut{}.mp4".format(i+1)) 
        clip.audio.write_audiofile(r"converted/converted{}.wav".format(i+1))
        os.remove(r"chunks/cut{}.mp4".format(i+1))

        model = whisper.load_model("base")
        result = model.transcribe("converted/converted{}.wav".format(i+1))
        os.remove(r"converted/converted{}.wav".format(i+1))
        print(round((i-emp)*100/len(rank),2),"%:")
        print(result["text"])
        diz['chunk{}'.format(i+1)]=result["text"]

        l_chunks=[str(i+1)+'\n00:'+"{:02d}".format(int(chunk_size*i/60))+':'+"{:02d}".format(chunk_size*i%60)+',000 --> '+'00:'+"{:02d}".format(int(chunk_size*(i+1)/60))+':'+"{:02d}".format(chunk_size*(i+1)%60-1)+',999\n'+diz['chunk{}'.format(i+1)] for i in range(min(rank),(len(diz)+min(rank)-1))]
        text='\n\n'.join(l_chunks)

        with open(filename[:-3]+'txt',mode ='w') as file:
          file.write("\n")
          file.write(text)
        
        p = Path(filename[:-3]+'txt')
        p.rename(p.with_suffix('.srt'))
    print("100%")
    print("Finally ready!")

## Get the video script
Function to get the complete video script.

In [4]:
def VideoScript(filename):
    clip = mp.VideoFileClip(filename)
    clip.audio.write_audiofile(filename[:-3]+".wav")
    model = whisper.load_model("base")
    result = model.transcribe(filename[:-3]+".wav")
    os.remove(filename[:-3]+".wav")
    Script=result["text"]
    with open('Script.txt',mode ='w') as file:
          file.write(Script)
    print(Script)
    return(Script)

## Create a summary from the script
Defines the functions necessary to summarize the Script into a maximum of max_len sentences

In [5]:
def AbstractiveSummarization(Script, max_len=5):
    sentences = Script.split(". ")
    textsize=len(sentences)
    paragraph_size=int(textsize/(max_len))
    paragraphs_lims=list(range(0,len(sentences)+1,paragraph_size))
    paragraphs_lims[-1]=textsize
    paragraphs=[]
    main_ideas=[]
    batch = tokenizer.prepare_seq2seq_batch(Script, truncation=True, padding='longest',return_tensors='pt')
    translated = model.generate(**batch)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    main_ideas.append(tgt_text)
    for i in range(1,len(paragraphs_lims)):
        paragraphs.append('. '.join(sentences[paragraphs_lims[i-1]:paragraphs_lims[i]]))
    for i in range(len(paragraphs)):
        print(min(round((i+1)*100/max_len,2),100),"%")
        batch = tokenizer.prepare_seq2seq_batch(paragraphs[i], truncation=True, padding='longest',return_tensors='pt')
        translated = model.generate(**batch)
        tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
        tgt_text[0] = tgt_text[0].replace('datememe','')
        tgt_text[0] = tgt_text[0].replace('DropCatch','')
        if 'n our series of' in tgt_text[0]:
            Names=re.findall(r"[A-Z][a-z]+,?\s+(?:[A-Z][a-z]*\.?\s*)?[A-Z][a-z]+", tgt_text[0])
            if len(Names)>0:
                presence=True
                for j in range(len(Names)):
                    if Names[j] not in paragraphs[i]:
                        presence=False
                if presence==False:
                    print("Incorrect summary: ", tgt_text[0])
                    print("Original text: ", paragraphs[i])
        main_ideas.append(tgt_text)
    return(''.join(idea[0] for idea in main_ideas))

Running the functions

The following function only creates the video subtitles in an SRT format to test the Whisper library

In [6]:
VideoToSRT("How Clicking a Single Link Can Cost Millions.mp4",5,0,180)

The video is 861 seconds
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
MoviePy - Writing audio in converted/converted1.wav


MoviePy - Done.


0.0 %:
 oh receive the function
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
MoviePy - Writing audio in converted/converted2.wav


MoviePy - Done.


2.7 %:
 Lloyd Fondghérad y Co Work Y Cienaden Mae'sol w街 o'n Ysd Eenig
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
MoviePy - Writing audio in converted/converted3.wav


...

91.89 %:
 a fe小 child ar g Champdgoes bod y ddimgnian yn bain i hyще yn elethau.
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
MoviePy - Writing audio in converted/converted36.wav


MoviePy - Done.


94.59 %:
 Wh Guard Goto Ye dau premia?
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
MoviePy - Writing audio in converted/converted37.wav


MoviePy - Done.


97.3 %:
 attacking MYlwyd ddim puzz Underground
100%
Finally ready!


The VideoScript function generates and prints the entire video script

In [7]:
ClickingMillions=VideoScript("How Clicking a Single Link Can Cost Millions.mp4")

MoviePy - Writing audio in How Clicking a Single Link Can Cost Millions..wav


chunk:   0%|          | 0/18981 [00:00<?, ?it/s, now=None]

MoviePy - Done.
 Whanau blaen ei dyna rwy 2021 Bo神eth Ys gaf frustrated a las Johannau Ckur Pm refugees eu nyw ar y Ys nhw y felly Mae confus, yr ysshac fy rya'r cyf古'r magyraeth a barod diwario eu ceithio Mae marowany wedi'r yser Cy getfaforlw'r a– y Fai Defektael Ys nhw at y aut exhaust sy nar dobrze chylu o eich arnu 계 Holdwch'n ni nhw i add i ku ffagDa butter Islandlation, ac ydw'n ringwant arher. Ond'r Intoidade jurdin ailwch att ni fi ac gwi'n ei'n jobsio llyrygu? Ond neu dwaudni holl. Aquel ei phiertraeth ymgσιw Somehow gust o ffur i'n digliwch gyrund reliedwaethir fydd di雫wr d cancelledi i pridleg sot mae ytty totu dde issuingど y dod badly wantedad en gyd i ffór. A ddim caelau ffawr hyn yn ymwyllaedead eiretr am ddiddir yw mewn urfanol ei gwneud darwm! Bdeld wneud traidydd yn ffaf resolvedig. Eu gynfal mae'r hynnyd gen gethkidos�� olai eu sylfa mewn ni-demryduddi. Cosa mewn mi gdrum a un Matru pegu peth i' mwy kalau fi dynaondi gwneud cyn un. Gallag'l gwlu yn cael ar fihing ool

# Summary

In [8]:
AbstractiveSummarization(ClickingMillions, 40)

2.5 %
5.0 %
7.5 %
10.0 %
12.5 %
15.0 %
17.5 %
20.0 %
22.5 %
25.0 %
27.5 %
30.0 %
32.5 %
35.0 %
37.5 %
40.0 %
42.5 %
45.0 %
47.5 %
50.0 %
52.5 %
55.0 %
57.5 %
60.0 %
62.5 %
65.0 %
67.5 %
70.0 %
72.5 %
75.0 %
77.5 %
80.0 %
82.5 %
85.0 %
87.5 %
90.0 %
92.5 %
95.0 %
97.5 %
100.0 %
100 %
100 %
100 %
100 %
100 %
100 %
100 %
100 %
100 %
100 %
100 %
100 %
100 %
100 %
100 %
100 %
100 %
100 %
100 %
100 %
100 %
100 %
100 %
100 %


''